In [1]:
# Create IMAM supervision data to upload into RapidPro

# Identify all supervision (State and LGA) and put in rows by site (name, phone, email)
# Merge State and LGA supervision into complete listing of personnel
# export as .xlsx file

In [3]:

import os


os.environ.setdefault("DJANGO_SETTINGS_MODULE", "IMAM.settings")

from django.conf import settings
from home.management.commands.load_data import assign_state_lga_num, rename_cols, generic_cleaning, merge_in_and_outpatients, add_program_reports_from_supervision

import django
django.setup()

from home.models import Site

In [42]:
# To show plots in the notebook
%matplotlib inline  

import pandas as pd
import pandas_highcharts.core
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot 

import os

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "IMAM.settings")

from django.conf import settings
from home.management.commands.load_data import rename_cols, assign_state_lga_num

import django
django.setup()

from home.models import First_admin, Second_admin, Site, Registration


In [51]:
engine = create_engine(
    'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**settings.DATABASES['default']))
df = pd.read_sql_query("select * from registration;", con=engine)

In [196]:
# If there is no data in PostGres
# df = pd.ExcelFile('/home/robert/Downloads/reg.xlsx').parse('Contacts')

In [50]:
df.head()

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail
0,ddb0c230-15d1-4964-a69f-2cc2e67ce864,tel:+2349033885605,Jummai A Wakataba.,,2.109110e+09,OTP,2016-11-24 11:01:11.431208,2017-04-24 08:14:10.664967,None,None
1,390d3f91-99e0-4fa7-9549-1b3d90322ba1,tel:+2348164132823,Talatu Wade.,,2.109110e+09,OTP,2016-11-09 08:49:41.987285,2017-04-24 08:14:09.783450,In Charge Hospital/PHC,None
2,75e7f34c-063d-4b1c-999e-58830bf40d5a,tel:+2349099115106,Adama Garba.,,2.108110e+09,OTP,2016-11-10 08:47:31.712254,2017-04-19 11:54:24.977057,In Charge Hospital/PHC,None
3,832194ca-e392-4c02-9fa5-19e67f77d0b7,tel:+2348066963335,Abdullahi Bilyaminu Taro.,,2.109110e+09,OTP,2016-11-09 08:59:42.150718,2017-04-24 08:14:07.162381,Community Health Officer,None
4,fc5204cf-e4f9-4a9f-b797-faf2e9e430e2,tel:+2348184880996,Iliya Saleh.,,2.108210e+09,SC,2016-11-11 08:39:43.985626,2017-04-24 08:14:06.376224,Nurse/Midwife,salehiliya@gmail.com


In [52]:
df.query('siteid!=siteid')

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail


In [7]:
# run rename columns function
rename_cols(df)

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail
0,ddb0c230-15d1-4964-a69f-2cc2e67ce864,tel:+2349033885605,Jummai A Wakataba.,,2.109110e+09,OTP,2016-11-24 11:01:11.431208,2017-04-24 08:14:10.664967,None,None
1,390d3f91-99e0-4fa7-9549-1b3d90322ba1,tel:+2348164132823,Talatu Wade.,,2.109110e+09,OTP,2016-11-09 08:49:41.987285,2017-04-24 08:14:09.783450,In Charge Hospital/PHC,None
2,75e7f34c-063d-4b1c-999e-58830bf40d5a,tel:+2349099115106,Adama Garba.,,2.108110e+09,OTP,2016-11-10 08:47:31.712254,2017-04-19 11:54:24.977057,In Charge Hospital/PHC,None
3,832194ca-e392-4c02-9fa5-19e67f77d0b7,tel:+2348066963335,Abdullahi Bilyaminu Taro.,,2.109110e+09,OTP,2016-11-09 08:59:42.150718,2017-04-24 08:14:07.162381,Community Health Officer,None
4,fc5204cf-e4f9-4a9f-b797-faf2e9e430e2,tel:+2348184880996,Iliya Saleh.,,2.108210e+09,SC,2016-11-11 08:39:43.985626,2017-04-24 08:14:06.376224,Nurse/Midwife,salehiliya@gmail.com
5,c419115c-4c15-48b2-b3ff-033631a2a669,tel:+2348163127206,Zuwaira Abubakar.,,2.109210e+09,SC,2016-11-23 10:20:53.773742,2017-04-24 08:14:15.651259,Community Health Officer,None
6,397644a3-0e74-4868-9203-7bd0bbd21f9a,tel:+2348086467626,Ismail Umar Libata.,,2.106310e+09,SC,2016-11-11 08:58:09.639373,2017-04-24 08:14:02.790182,Nurse/Midwife,iulibata@yahoo.com
7,ea9fbb08-3f83-4d49-92bb-9827391fa836,tel:+2348132096085,Dimka John Yakubu.,,2.106310e+09,SC,2016-11-11 09:00:03.567239,2017-04-24 08:14:03.617632,Doctor,johndimka@yahoo.co.uk
8,f240f3af-78fe-4716-9b6a-1a72bbcdfb6b,tel:+2348188835472,Aminu Usman G. .,,2.108110e+09,OTP,2016-11-10 08:47:54.587866,2017-04-19 11:54:23.388718,LabTech-Pharm,None
9,2e75a5b0-9685-4f68-bbad-ed00ab925a21,tel:+2348063125298,Maryam Abdullahi.,,2.106110e+09,OTP,2016-11-12 08:21:06.024316,2017-04-26 08:11:47.603919,Community Health Officer,None


In [8]:
# rename SiteID to siteid
# df=df.rename(columns = {'SiteID':'siteid'})

# Change the order (the index) of the columns
columnsTitles = ['siteid',                 
                 'name',
                 'urn',
                 'mail',
                 'post',
                 'type',
                 'groups',
                 'first_seen',
                 'last_seen'
                 ]

df2 = df.reindex(columns=columnsTitles)
df2

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen
0,2.109110e+09,Jummai A Wakataba.,tel:+2349033885605,None,None,OTP,,2016-11-24 11:01:11.431208,2017-04-24 08:14:10.664967
1,2.109110e+09,Talatu Wade.,tel:+2348164132823,None,In Charge Hospital/PHC,OTP,,2016-11-09 08:49:41.987285,2017-04-24 08:14:09.783450
2,2.108110e+09,Adama Garba.,tel:+2349099115106,None,In Charge Hospital/PHC,OTP,,2016-11-10 08:47:31.712254,2017-04-19 11:54:24.977057
3,2.109110e+09,Abdullahi Bilyaminu Taro.,tel:+2348066963335,None,Community Health Officer,OTP,,2016-11-09 08:59:42.150718,2017-04-24 08:14:07.162381
4,2.108210e+09,Iliya Saleh.,tel:+2348184880996,salehiliya@gmail.com,Nurse/Midwife,SC,,2016-11-11 08:39:43.985626,2017-04-24 08:14:06.376224
5,2.109210e+09,Zuwaira Abubakar.,tel:+2348163127206,None,Community Health Officer,SC,,2016-11-23 10:20:53.773742,2017-04-24 08:14:15.651259
6,2.106310e+09,Ismail Umar Libata.,tel:+2348086467626,iulibata@yahoo.com,Nurse/Midwife,SC,,2016-11-11 08:58:09.639373,2017-04-24 08:14:02.790182
7,2.106310e+09,Dimka John Yakubu.,tel:+2348132096085,johndimka@yahoo.co.uk,Doctor,SC,,2016-11-11 09:00:03.567239,2017-04-24 08:14:03.617632
8,2.108110e+09,Aminu Usman G. .,tel:+2348188835472,None,LabTech-Pharm,OTP,,2016-11-10 08:47:54.587866,2017-04-19 11:54:23.388718
9,2.106110e+09,Maryam Abdullahi.,tel:+2348063125298,None,Community Health Officer,OTP,,2016-11-12 08:21:06.024316,2017-04-26 08:11:47.603919


In [9]:
assign_state_lga_num(df2)

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num
0,2.109110e+09,Jummai A Wakataba.,tel:+2349033885605,None,None,OTP,,2016-11-24 11:01:11.431208,2017-04-24 08:14:10.664967,12,21,2109
1,2.109110e+09,Talatu Wade.,tel:+2348164132823,None,In Charge Hospital/PHC,OTP,,2016-11-09 08:49:41.987285,2017-04-24 08:14:09.783450,12,21,2109
2,2.108110e+09,Adama Garba.,tel:+2349099115106,None,In Charge Hospital/PHC,OTP,,2016-11-10 08:47:31.712254,2017-04-19 11:54:24.977057,12,21,2108
3,2.109110e+09,Abdullahi Bilyaminu Taro.,tel:+2348066963335,None,Community Health Officer,OTP,,2016-11-09 08:59:42.150718,2017-04-24 08:14:07.162381,12,21,2109
4,2.108210e+09,Iliya Saleh.,tel:+2348184880996,salehiliya@gmail.com,Nurse/Midwife,SC,,2016-11-11 08:39:43.985626,2017-04-24 08:14:06.376224,12,21,2108
5,2.109210e+09,Zuwaira Abubakar.,tel:+2348163127206,None,Community Health Officer,SC,,2016-11-23 10:20:53.773742,2017-04-24 08:14:15.651259,12,21,2109
6,2.106310e+09,Ismail Umar Libata.,tel:+2348086467626,iulibata@yahoo.com,Nurse/Midwife,SC,,2016-11-11 08:58:09.639373,2017-04-24 08:14:02.790182,12,21,2106
7,2.106310e+09,Dimka John Yakubu.,tel:+2348132096085,johndimka@yahoo.co.uk,Doctor,SC,,2016-11-11 09:00:03.567239,2017-04-24 08:14:03.617632,12,21,2106
8,2.108110e+09,Aminu Usman G. .,tel:+2348188835472,None,LabTech-Pharm,OTP,,2016-11-10 08:47:54.587866,2017-04-19 11:54:23.388718,12,21,2108
9,2.106110e+09,Maryam Abdullahi.,tel:+2348063125298,None,Community Health Officer,OTP,,2016-11-12 08:21:06.024316,2017-04-26 08:11:47.603919,12,21,2106


In [10]:
# Create back-up
restore = df2

In [11]:
# Restore
df2 = restore

In [12]:
df2

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num
0,2.109110e+09,Jummai A Wakataba.,tel:+2349033885605,None,None,OTP,,2016-11-24 11:01:11.431208,2017-04-24 08:14:10.664967,12,21,2109
1,2.109110e+09,Talatu Wade.,tel:+2348164132823,None,In Charge Hospital/PHC,OTP,,2016-11-09 08:49:41.987285,2017-04-24 08:14:09.783450,12,21,2109
2,2.108110e+09,Adama Garba.,tel:+2349099115106,None,In Charge Hospital/PHC,OTP,,2016-11-10 08:47:31.712254,2017-04-19 11:54:24.977057,12,21,2108
3,2.109110e+09,Abdullahi Bilyaminu Taro.,tel:+2348066963335,None,Community Health Officer,OTP,,2016-11-09 08:59:42.150718,2017-04-24 08:14:07.162381,12,21,2109
4,2.108210e+09,Iliya Saleh.,tel:+2348184880996,salehiliya@gmail.com,Nurse/Midwife,SC,,2016-11-11 08:39:43.985626,2017-04-24 08:14:06.376224,12,21,2108
5,2.109210e+09,Zuwaira Abubakar.,tel:+2348163127206,None,Community Health Officer,SC,,2016-11-23 10:20:53.773742,2017-04-24 08:14:15.651259,12,21,2109
6,2.106310e+09,Ismail Umar Libata.,tel:+2348086467626,iulibata@yahoo.com,Nurse/Midwife,SC,,2016-11-11 08:58:09.639373,2017-04-24 08:14:02.790182,12,21,2106
7,2.106310e+09,Dimka John Yakubu.,tel:+2348132096085,johndimka@yahoo.co.uk,Doctor,SC,,2016-11-11 09:00:03.567239,2017-04-24 08:14:03.617632,12,21,2106
8,2.108110e+09,Aminu Usman G. .,tel:+2348188835472,None,LabTech-Pharm,OTP,,2016-11-10 08:47:54.587866,2017-04-19 11:54:23.388718,12,21,2108
9,2.106110e+09,Maryam Abdullahi.,tel:+2348063125298,None,Community Health Officer,OTP,,2016-11-12 08:21:06.024316,2017-04-26 08:11:47.603919,12,21,2106


In [13]:
# What is the type of siteid
print type(df2['siteid'])
# This does not tell us anything about the variable

<class 'pandas.core.series.Series'>


In [19]:
df2['siteid'].describe()
# previously this would give the type of the var. 
# now referring to object

count    2.568000e+03
mean     1.741722e+09
std      1.268128e+09
min      2.000000e+00
25%      5.131100e+08
50%      1.921110e+09
75%      3.304435e+09
max      3.613210e+09
Name: siteid, dtype: float64

In [92]:
df2['siteid'] = pd.to_numeric(df2['siteid'], errors='coerce')

In [15]:
# Remove NaN from siteid

# If you don't specify a variable this will drop any rows including a NaN
df2 = df2.dropna(subset=['siteid'])

In [16]:
df2 = df2.query('siteid<3999990999')

In [17]:
df2 = df2.query('siteid>1')


In [18]:
df2 = df2.query('siteid!=99')

In [141]:
# This is supposed to remove non-numeric chars from siteid
# df2['siteid'] = filter(lambda x: x.isdigit(), df2['siteid'])

# for i in ('weeknum', 'state_num', 'lga_num', 'siteid'):
#     # IDs convert from string to float
#     dataframe[i] = pd.to_numeric(dataframe[i], errors='coerce')
#     # Clean out of range identification data - this deletes entire row where a NaN is found
#     dataframe = dataframe.query('%s==%s' % (i, i)).query('%s>=0' % i)
#     # Convert from float to int
#     dataframe[i] = dataframe[i].astype(int)


In [20]:
df2['siteid'] = df2['siteid'].astype(int)

In [21]:
df2['siteid'].describe()

count    2.568000e+03
mean     1.741722e+09
std      1.268128e+09
min      2.000000e+00
25%      5.131100e+08
50%      1.921110e+09
75%      3.304435e+09
max      3.613210e+09
Name: siteid, dtype: float64

In [22]:
df2['siteid'].value_counts()


19            18
35            11
21            10
20            10
33             9
2              8
16             8
3503210022     8
36             7
5              7
2106110004     7
3511110005     7
3512110004     7
2106110020     7
2118110015     6
3306110017     6
3603110029     6
1703110012     6
3305110027     6
3513110005     6
1714110008     6
2109210028     5
3512           5
2001110059     5
3301110011     5
1940110023     5
2118110014     5
2104110014     5
2112110009     5
3321110020     5
              ..
2109110019     1
217            1
819110013      1
827110010      1
824110002      1
3613210035     1
3505           1
2001110001     1
2121210012     1
2004110035     1
3508110034     1
818118813      1
3508110020     1
3506           1
1717           1
1719           1
3504           1
1727           1
2108110005     1
1703310004     1
201            1
3502           1
203            1
3301           1
3305           1
209            1
2010110006     1
213           

In [63]:
# Remove erroneous SiteIDs from df

#df2 = df2.query('1>siteid>3799990999')

#dataframe = dataframe.query('101110001<siteid<3799990999')

In [23]:
df2.siteid.unique()

array([2109110019, 2109110016, 2108110007, ..., 2111110007, 2111110006,
        211110047])

In [179]:
# Convert siteid to number
# df2['siteid'] = pd.to_numeric(df2.siteid, errors='coerce', downcast='signed').astype(int)


In [24]:
# create db with only supervision staff
supervision_df = df2[df2['siteid'] <= 3699]
# supervision siteids range from 1 to 3699

# Sort data
supervision_df = supervision_df.sort_values(by='siteid')
supervision_df

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num
1875,2,Wullanga Alfred,tel:+2347032853473,wulangaalfred@gmail.com,Coordinator,None,,2016-10-26 09:06:02.356386,2017-04-24 08:35:34.294050,3,2.,2.0
1877,2,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Coordinator,None,,2016-10-26 09:04:19.462503,2017-04-24 08:35:35.817844,3,2.,2.0
1878,2,Ijagila Mark .,tel:+2348100001300,ijagilamark@gmail.com,Coordinator,None,,2016-10-24 09:36:56.209869,2017-04-24 08:35:36.598891,3,2.,2.0
1873,2,Reuben Aidaticha.,tel:+2348038213859,aidaticha@gmail.com,Database Manager,None,,2016-10-24 09:35:36.024432,2017-04-24 08:35:38.078705,3,2.,2.0
1872,2,Hauwa Zoakah.,tel:+2347035678763,hauwabata@yahoo.com,Coordinator,None,,2016-06-01 12:17:48.149140,2017-04-24 08:35:37.670118,3,2.,2.0
1871,2,Wullanga Alfred.,tel:+2348029864318,wullangaalfred@gmail.com,Database Manager,None,,2016-10-24 09:35:46.229729,2017-04-24 08:35:38.481902,3,2.,2.0
1876,2,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Database Manager,None,,2016-10-24 10:38:14.041832,2017-04-24 08:35:37.132138,3,2.,2.0
1874,2,Olawumi Monica Ajayi.,tel:+2348036173901,wumi.ajayi@yahoo.com,Technical Assistance,None,,2016-10-24 09:38:19.635591,2017-04-24 08:35:35.056843,3,2.,2.0
1009,5,Sama'ila Usman Maikan.,tel:+2348038233464,None,Coordinator,None,,2016-09-02 10:42:56.346388,2017-02-27 11:33:31.642024,3,5.,5.0
986,5,Jackson Ladu Martins.,tel:+2348035350951,jmartins@unicef.org,Technical Assistance,None,,2016-09-02 10:40:28.426741,2017-02-27 11:33:29.753410,3,5.,5.0


In [112]:
# 13 states and 282 LGA = 294 total supervision sites
# Are there CMAM programs in all LGAs? 
supervision_df['siteid'].value_counts()

19      18
35      11
20      10
21      10
33       9
2        8
16       8
5        7
36       7
18       5
3512     5
17       5
1905     4
2016     4
3503     4
3313     4
3507     4
8        4
1823     4
2106     4
2011     4
3515     3
1703     3
3304     3
3306     3
202      3
3611     3
3308     3
813      3
2112     3
        ..
2001     1
2007     1
2010     1
2024     1
2033     1
819      1
814      1
1727     1
220      1
201      1
203      1
206      1
209      1
213      1
215      1
216      1
217      1
219      1
2034     1
812      1
3301     1
3305     1
3319     1
3321     1
802      1
803      1
804      1
808      1
809      1
512      1
Name: siteid, dtype: int64

In [25]:
# How many registrations in supervision ? 
len(supervision_df['siteid'])

330

In [114]:
# Many supervision staff did not record their post - should recode them all as supervisors
supervision_df['post'].str.upper().value_counts()
# str.upper() does not make permanent change. 

# I think there was a data entry error with Post because most people entered their post correctly
# also there should not be any HC, HW in the supervision cadres

COORDINATOR             196
TECHNICAL ASSISTANCE     53
STOCKS MANAGER           33
DATABASE MANAGER         27
OBSERVER                  5
S                         2
D                         2
Name: post, dtype: int64

In [26]:
# to pivot data, create counts of each case by siteid

# df['count'] = df.groupby('col').cumcount() + 1
supervision_df['count'] = supervision_df.groupby('siteid').cumcount() + 1

In [27]:
supervision_df

,siteid,name,urn,mail,post,type,groups,first_seen,last_seen,siteid_lgt,state_num,lga_num,count
1875,2,Wullanga Alfred,tel:+2347032853473,wulangaalfred@gmail.com,Coordinator,None,,2016-10-26 09:06:02.356386,2017-04-24 08:35:34.294050,3,2.,2.0,1
1877,2,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Coordinator,None,,2016-10-26 09:04:19.462503,2017-04-24 08:35:35.817844,3,2.,2.0,2
1878,2,Ijagila Mark .,tel:+2348100001300,ijagilamark@gmail.com,Coordinator,None,,2016-10-24 09:36:56.209869,2017-04-24 08:35:36.598891,3,2.,2.0,3
1873,2,Reuben Aidaticha.,tel:+2348038213859,aidaticha@gmail.com,Database Manager,None,,2016-10-24 09:35:36.024432,2017-04-24 08:35:38.078705,3,2.,2.0,4
1872,2,Hauwa Zoakah.,tel:+2347035678763,hauwabata@yahoo.com,Coordinator,None,,2016-06-01 12:17:48.149140,2017-04-24 08:35:37.670118,3,2.,2.0,5
1871,2,Wullanga Alfred.,tel:+2348029864318,wullangaalfred@gmail.com,Database Manager,None,,2016-10-24 09:35:46.229729,2017-04-24 08:35:38.481902,3,2.,2.0,6
1876,2,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Database Manager,None,,2016-10-24 10:38:14.041832,2017-04-24 08:35:37.132138,3,2.,2.0,7
1874,2,Olawumi Monica Ajayi.,tel:+2348036173901,wumi.ajayi@yahoo.com,Technical Assistance,None,,2016-10-24 09:38:19.635591,2017-04-24 08:35:35.056843,3,2.,2.0,8
1009,5,Sama'ila Usman Maikan.,tel:+2348038233464,None,Coordinator,None,,2016-09-02 10:42:56.346388,2017-02-27 11:33:31.642024,3,5.,5.0,1
986,5,Jackson Ladu Martins.,tel:+2348035350951,jmartins@unicef.org,Technical Assistance,None,,2016-09-02 10:40:28.426741,2017-02-27 11:33:29.753410,3,5.,5.0,2


In [31]:
# supervision_df.pivot(columns=['name','urn','mail'], index='siteid')
# error

In [28]:
# Three data points to include in IMAM Supervision database for each supervision SiteID
# Name, Phone Number (URN), email

columnsTitles = ['siteid',                 
                 'name',
                 'urn',
                 'mail',
                 'count',
                 ]

supervision_df = supervision_df.reindex(columns=columnsTitles)

In [29]:
# create db with only State Level supervision staff
state_df = supervision_df[supervision_df['siteid'] <= 39]


In [31]:
state_df

,siteid,name,urn,mail,count
1875,2,Wullanga Alfred,tel:+2347032853473,wulangaalfred@gmail.com,1
1877,2,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,2
1878,2,Ijagila Mark .,tel:+2348100001300,ijagilamark@gmail.com,3
1873,2,Reuben Aidaticha.,tel:+2348038213859,aidaticha@gmail.com,4
1872,2,Hauwa Zoakah.,tel:+2347035678763,hauwabata@yahoo.com,5
1871,2,Wullanga Alfred.,tel:+2348029864318,wullangaalfred@gmail.com,6
1876,2,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,7
1874,2,Olawumi Monica Ajayi.,tel:+2348036173901,wumi.ajayi@yahoo.com,8
1009,5,Sama'ila Usman Maikan.,tel:+2348038233464,None,1
986,5,Jackson Ladu Martins.,tel:+2348035350951,jmartins@unicef.org,2


In [32]:
# convert vertical to horizontal database. 
# will None in the cells cause us to send excessive number of warning SMS? 
state_wide = state_df.pivot(index='siteid', columns='count')

In [33]:
state_wide.head()

# replaced multiIndex with one row of column names

name                                              \
count                         1                       2                   3    
siteid                                                                         
2                Wullanga Alfred          Hauwa Zoakah .      Ijagila Mark .   
5         Sama'ila Usman Maikan.   Jackson Ladu Martins.        Yakubu Baba.   
8       Hassana Suleiman Jibrin.  Abdullahi Alhaji Madi.      Daniel James .   
16           Ibrahim Inuwa Lano.        Suleiman Mamman.  Ronas Amos Amusa .   
17        Temidayo Esther Ajala.  Shuaibu Aliyu Ringim .  Olatomiwa Olabisi.   

                                                             \
count                         4                          5    
siteid                                                        
2              Reuben Aidaticha.              Hauwa Zoakah.   
5       Charity Evans Nysalamke.     Habu Abdulmalik Dauda.   
8           Aminu Usman Danzomo.                       None   
16                 Usman Baraya.          Selamawit Negash.   
17             Saidu Umar Adamu.  MUSA  MOHAMMED  HADEJIA .   

                                                             \
count                               6                    7    
siteid                                                        
2                     Wullanga Alfred.    Reuben Aidaticha.   
5                  Hamza Yakubu Sade .     Ali Shehu Kobi .   
8                                 None                 None   
16      Olufunmilayo Adepoju-adebambo.  Ahmed Audu Saddana.   
17                                None                 None   

                                           ...   mail                          \
count                      8     9     10  ...     9     10    11    12    13   
siteid                                     ...                                  
2       Olawumi Monica Ajayi.  None  None  ...   None  None  None  None  None   
5                        None  None  None  ...   None  None  None  None  None   
8                        None  None  None  ...   None  None  None  None  None   
16             Rukayya Lawal.  None  None  ...   None  None  None  None  None   
17                       None  None  None  ...   None  None  None  None  None   

                                      
count     14    15    16    17    18  
siteid                                
2       None  None  None  None  None  
5       None  None  None  None  None  
8       None  None  None  None  None  
16      None  None  None  None  None  
17      None  None  None  None  None  

[5 rows x 54 columns]

In [59]:
[x for x in state_wide.columns]

[('name', 1),
 ('name', 2),
 ('name', 3),
 ('name', 4),
 ('name', 5),
 ('name', 6),
 ('name', 7),
 ('name', 8),
 ('name', 9),
 ('name', 10),
 ('name', 11),
 ('name', 12),
 ('name', 13),
 ('name', 14),
 ('name', 15),
 ('name', 16),
 ('name', 17),
 ('name', 18),
 ('urn', 1),
 ('urn', 2),
 ('urn', 3),
 ('urn', 4),
 ('urn', 5),
 ('urn', 6),
 ('urn', 7),
 ('urn', 8),
 ('urn', 9),
 ('urn', 10),
 ('urn', 11),
 ('urn', 12),
 ('urn', 13),
 ('urn', 14),
 ('urn', 15),
 ('urn', 16),
 ('urn', 17),
 ('urn', 18),
 ('mail', 1),
 ('mail', 2),
 ('mail', 3),
 ('mail', 4),
 ('mail', 5),
 ('mail', 6),
 ('mail', 7),
 ('mail', 8),
 ('mail', 9),
 ('mail', 10),
 ('mail', 11),
 ('mail', 12),
 ('mail', 13),
 ('mail', 14),
 ('mail', 15),
 ('mail', 16),
 ('mail', 17),
 ('mail', 18)]

In [61]:
# Create new column name
state_wide.columns = ["State" + (state_wide.columns[i][0]) + str(state_wide.columns[i][1]) for i in range(len(state_wide.columns))]

In [63]:
state_wide.reset_index()

,siteid,Statename1,Statename2,Statename3,Statename4,Statename5,Statename6,Statename7,Statename8,Statename9,...,Statemail9,Statemail10,Statemail11,Statemail12,Statemail13,Statemail14,Statemail15,Statemail16,Statemail17,Statemail18
0,2,Wullanga Alfred,Hauwa Zoakah .,Ijagila Mark .,Reuben Aidaticha.,Hauwa Zoakah.,Wullanga Alfred.,Reuben Aidaticha.,Olawumi Monica Ajayi.,None,...,None,None,None,None,None,None,None,None,None,None
1,5,Sama'ila Usman Maikan.,Jackson Ladu Martins.,Yakubu Baba.,Charity Evans Nysalamke.,Habu Abdulmalik Dauda.,Hamza Yakubu Sade .,Ali Shehu Kobi .,None,None,...,None,None,None,None,None,None,None,None,None,None
2,8,Hassana Suleiman Jibrin.,Abdullahi Alhaji Madi.,Daniel James .,Aminu Usman Danzomo.,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,16,Ibrahim Inuwa Lano.,Suleiman Mamman.,Ronas Amos Amusa .,Usman Baraya.,Selamawit Negash.,Olufunmilayo Adepoju-adebambo.,Ahmed Audu Saddana.,Rukayya Lawal.,None,...,None,None,None,None,None,None,None,None,None,None
4,17,Temidayo Esther Ajala.,Shuaibu Aliyu Ringim .,Olatomiwa Olabisi.,Saidu Umar Adamu.,MUSA MOHAMMED HADEJIA .,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,18,Jane Gwani.,Saratu Aduwak.,Florence Ebun Oni.,Hauwa Usman .,Maryam Yusuf .,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,19,Ado Jibrin Sanda.,Akilu Sani.,Olalekan Wahab Runmonkun .,Ayodeji Osunkentan.,Nyako William Azi.,Abigail Ishaya Nyam .,Ado Mustapha.,Deborah Abi Nyako.,Sabo Wada.,...,sabowada57@gmail.com,ffekura@gmail.com,None,mikekush70@yahoo.com,akilumfashi@gmail.com,abolarin.samuelsesan@gmail.com,sabowada57@gmail.com,dgauji@yahoo.com,murtalamuhd33@gmail.com,msaniabdullahi@gmail.com
7,20,Hamisu Idris K/bai.,Zayyana Isyaku Sule.,Ibrahim Maje Sayyadi.,Yusufahmadu Gangara.,Abdulmumin Lawan.,Sim Karla Sakyenu.,Musbahu Hamisu.,Abdulhadi Abdulkadir.,Rabia Mohammed Sno .,...,rabiadaura77@yahoo.com,ibrado30@gmail.com,None,None,None,None,None,None,None,None
8,21,Beatrice Kwere.,Suwaiba M Bello.,Abdulmalik Muhammad Illo .,Sunday Norbert.,Aliyu Galadima Libata .,Abimbola Abosede Sobo.,Abisola Mary Atoyebi.,Shamsu Muhammed.,Aliyu Galadima Libata.,...,kabanjali@yahoo.com.hk,None,None,None,None,None,None,None,None,None
9,30,30.,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [41]:
df = state_wide
df.columns = [''.join(col).strip() for col in df.columns.values]

TypeError: sequence item 1: expected string, int found

In [170]:
# Rename column headers - multiIndex
# need to have only one line with column names at top of df

level_o = state_wide.columns.tolist()
columnsTitles =[]
for item in level_o:
    columnsTitles.append((item[0],"State"+item[0].capitalize()+str(item[1])))

print columnsTitles

[('name', 'StateName1'), ('name', 'StateName2'), ('name', 'StateName3'), ('name', 'StateName4'), ('name', 'StateName5'), ('name', 'StateName6'), ('name', 'StateName7'), ('name', 'StateName8'), ('name', 'StateName9'), ('name', 'StateName10'), ('name', 'StateName11'), ('name', 'StateName12'), ('name', 'StateName13'), ('name', 'StateName14'), ('name', 'StateName15'), ('name', 'StateName16'), ('name', 'StateName17'), ('name', 'StateName18'), ('urn', 'StateUrn1'), ('urn', 'StateUrn2'), ('urn', 'StateUrn3'), ('urn', 'StateUrn4'), ('urn', 'StateUrn5'), ('urn', 'StateUrn6'), ('urn', 'StateUrn7'), ('urn', 'StateUrn8'), ('urn', 'StateUrn9'), ('urn', 'StateUrn10'), ('urn', 'StateUrn11'), ('urn', 'StateUrn12'), ('urn', 'StateUrn13'), ('urn', 'StateUrn14'), ('urn', 'StateUrn15'), ('urn', 'StateUrn16'), ('urn', 'StateUrn17'), ('urn', 'StateUrn18'), ('mail', 'StateMail1'), ('mail', 'StateMail2'), ('mail', 'StateMail3'), ('mail', 'StateMail4'), ('mail', 'StateMail5'), ('mail', 'StateMail6'), ('mail', 

In [175]:
#Rename multiIndex

# want to have variable names with 
# - Level = State
# - Variable type = Name
# - Running number = 1 - 20
# Normally there are only 3 or 4 persons in supervision cadre at state level

corrected_varnames = state_wide.reindex(columns=columnsTitles)
# maybe should only rename columns not reindex here. 
# Use MultiIndex.rename ?

corrected_varnames
# why are all data are deleted ?
# http://stackoverflow.com/questions/34439437/how-to-reindex-a-a-pandas-dataframe-by-date-range-without-deleting-values

name                                                         \
count  StateName1 StateName2 StateName3 StateName4 StateName5 StateName6   
siteid                                                                     
2             NaN        NaN        NaN        NaN        NaN        NaN   
5             NaN        NaN        NaN        NaN        NaN        NaN   
8             NaN        NaN        NaN        NaN        NaN        NaN   
16            NaN        NaN        NaN        NaN        NaN        NaN   
17            NaN        NaN        NaN        NaN        NaN        NaN   
18            NaN        NaN        NaN        NaN        NaN        NaN   
19            NaN        NaN        NaN        NaN        NaN        NaN   
20            NaN        NaN        NaN        NaN        NaN        NaN   
21            NaN        NaN        NaN        NaN        NaN        NaN   
30            NaN        NaN        NaN        NaN        NaN        NaN   
33            NaN        NaN        NaN        NaN        NaN        NaN   
35            NaN        NaN        NaN        NaN        NaN        NaN   
36            NaN        NaN        NaN        NaN        NaN        NaN   

                                                        ...           mail  \
count  StateName7 StateName8 StateName9 StateName10     ...     StateMail9   
siteid                                                  ...                  
2             NaN        NaN        NaN         NaN     ...            NaN   
5             NaN        NaN        NaN         NaN     ...            NaN   
8             NaN        NaN        NaN         NaN     ...            NaN   
16            NaN        NaN        NaN         NaN     ...            NaN   
17            NaN        NaN        NaN         NaN     ...            NaN   
18            NaN        NaN        NaN         NaN     ...            NaN   
19            NaN        NaN        NaN         NaN     ...            NaN   
20            NaN        NaN        NaN         NaN     ...            NaN   
21            NaN        NaN        NaN         NaN     ...            NaN   
30            NaN        NaN        NaN         NaN     ...            NaN   
33            NaN        NaN        NaN         NaN     ...            NaN   
35            NaN        NaN        NaN         NaN     ...            NaN   
36            NaN        NaN        NaN         NaN     ...            NaN   

                                                                    \
count  StateMail10 StateMail11 StateMail12 StateMail13 StateMail14   
siteid                                                               
2              NaN         NaN         NaN         NaN         NaN   
5              NaN         NaN         NaN         NaN         NaN   
8              NaN         NaN         NaN         NaN         NaN   
16             NaN         NaN         NaN         NaN         NaN   
17             NaN         NaN         NaN         NaN         NaN   
18             NaN         NaN         NaN         NaN         NaN   
19             NaN         NaN         NaN         NaN         NaN   
20             NaN         NaN         NaN         NaN         NaN   
21             NaN         NaN         NaN         NaN         NaN   
30             NaN         NaN         NaN         NaN         NaN   
33             NaN         NaN         NaN         NaN         NaN   
35             NaN         NaN         NaN         NaN         NaN   
36             NaN         NaN         NaN         NaN         NaN   

                                                        
count  StateMail15 StateMail16 StateMail17 StateMail18  
siteid                                                  
2              NaN         NaN         NaN         NaN  
5              NaN         NaN         NaN         NaN  
8              NaN         NaN         NaN         NaN  
16             NaN         NaN         NaN         NaN  
17             NaN         

In [134]:
# Drop first row of index
corrected_varnames.columns = corrected_varnames.columns.droplevel(0)
corrected_varnames

count,StateName1,StateName2,StateName3,StateName4,StateName5,StateName6,StateName7,StateName8,StateName9,StateName10,...,StateMail9,StateMail10,StateMail11,StateMail12,StateMail13,StateMail14,StateMail15,StateMail16,StateMail17,StateMail18
siteid,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
corrected_varnames.reset_index()

count,siteid,StateName1,StateName2,StateName3,StateName4,StateName5,StateName6,StateName7,StateName8,StateName9,...,StateMail9,StateMail10,StateMail11,StateMail12,StateMail13,StateMail14,StateMail15,StateMail16,StateMail17,StateMail18
0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Try a different method
# Try to create list of column names in one row and change the names of first row - var names

In [216]:
level_o = state_wide.columns.tolist()
names =[]
for item in level_o:
    names.append(("State"+item[0].capitalize()+str(item[1])))

print names

['StateName1', 'StateName2', 'StateName3', 'StateName4', 'StateName5', 'StateName6', 'StateName7', 'StateName8', 'StateName9', 'StateName10', 'StateName11', 'StateName12', 'StateName13', 'StateName14', 'StateName15', 'StateName16', 'StateName17', 'StateName18', 'StateUrn1', 'StateUrn2', 'StateUrn3', 'StateUrn4', 'StateUrn5', 'StateUrn6', 'StateUrn7', 'StateUrn8', 'StateUrn9', 'StateUrn10', 'StateUrn11', 'StateUrn12', 'StateUrn13', 'StateUrn14', 'StateUrn15', 'StateUrn16', 'StateUrn17', 'StateUrn18', 'StateMail1', 'StateMail2', 'StateMail3', 'StateMail4', 'StateMail5', 'StateMail6', 'StateMail7', 'StateMail8', 'StateMail9', 'StateMail10', 'StateMail11', 'StateMail12', 'StateMail13', 'StateMail14', 'StateMail15', 'StateMail16', 'StateMail17', 'StateMail18']


In [203]:
state_wide.columns = state_wide.columns.droplevel(0)
state_wide

count,1,2,3,4,5,6,7,8,9,10,...,9,10,11,12,13,14,15,16,17,18
siteid,,,,,,,,,,,,,,,,,,,,,
2,Wullanga Alfred,Hauwa Zoakah .,Ijagila Mark .,Reuben Aidaticha.,Hauwa Zoakah.,Wullanga Alfred.,Reuben Aidaticha.,Olawumi Monica Ajayi.,None,None,...,None,None,None,None,None,None,None,None,None,None
5,Sama'ila Usman Maikan.,Jackson Ladu Martins.,Yakubu Baba.,Charity Evans Nysalamke.,Habu Abdulmalik Dauda.,Hamza Yakubu Sade .,Ali Shehu Kobi .,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,Hassana Suleiman Jibrin.,Abdullahi Alhaji Madi.,Daniel James .,Aminu Usman Danzomo.,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
16,Ibrahim Inuwa Lano.,Suleiman Mamman.,Ronas Amos Amusa .,Usman Baraya.,Selamawit Negash.,Olufunmilayo Adepoju-adebambo.,Ahmed Audu Saddana.,Rukayya Lawal.,None,None,...,None,None,None,None,None,None,None,None,None,None
17,Temidayo Esther Ajala.,Shuaibu Aliyu Ringim .,Olatomiwa Olabisi.,Saidu Umar Adamu.,MUSA MOHAMMED HADEJIA .,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
18,Jane Gwani.,Saratu Aduwak.,Florence Ebun Oni.,Hauwa Usman .,Maryam Yusuf .,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
19,Ado Jibrin Sanda.,Akilu Sani.,Olalekan Wahab Runmonkun .,Ayodeji Osunkentan.,Nyako William Azi.,Abigail Ishaya Nyam .,Ado Mustapha.,Deborah Abi Nyako.,Sabo Wada.,Fumen Fuset Emmanuel.,...,sabowada57@gmail.com,ffekura@gmail.com,None,mikekush70@yahoo.com,akilumfashi@gmail.com,abolarin.samuelsesan@gmail.com,sabowada57@gmail.com,dgauji@yahoo.com,murtalamuhd33@gmail.com,msaniabdullahi@gmail.com
20,Hamisu Idris K/bai.,Zayyana Isyaku Sule.,Ibrahim Maje Sayyadi.,Yusufahmadu Gangara.,Abdulmumin Lawan.,Sim Karla Sakyenu.,Musbahu Hamisu.,Abdulhadi Abdulkadir.,Rabia Mohammed Sno .,Ado Ibrahim Abdulrahman.,...,rabiadaura77@yahoo.com,ibrado30@gmail.com,None,None,None,None,None,None,None,None
21,Beatrice Kwere.,Suwaiba M Bello.,Abdulmalik Muhammad Illo .,Sunday Norbert.,Aliyu Galadima Libata .,Abimbola Abosede Sobo.,Abisola Mary Atoyebi.,Shamsu Muhammed.,Aliyu Galadima Libata.,Abdulmalik Muhammad Illo.,...,kabanjali@yahoo.com.hk,None,None,None,None,None,None,None,None,None


In [217]:
# how many elements in list
len(names)

54

In [218]:
state_wide.rename(index=str, columns=([names])

SyntaxError: unexpected EOF while parsing (<ipython-input-218-412b6bf68cb1>, line 1)

In [196]:
state_wide.reindex(columns=names)

ValueError: cannot reindex from a duplicate axis

In [ ]:
pd.unique(state_df.name.ravel())
# Ravel - Return the flattened underlying data as an ndarray